1. Unpack both parts of the Yahoo dataset
2. Join them together (execute in command line / terminal): cat FUllOct2007.xml.part1 FullOct2007.xml.part2 > full.xml
3. Install xml_split: sudo apt-get install xml-twig-tools
4. Split the full file into approx. 1Gb big files: xml_split -s 1Gb full.xml

In [14]:
!mkdir Webscope
!tar -xvzf dataset-1.tgz -C Webscope && tar -xvzf dataset-2.tgz -C Webscope
!cd Webscope && gunzip *.gz && pwd

mkdir: cannot create directory ‘Webscope’: File exists
tar (child): dataset-1.tgz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now
gzip: *.gz: No such file or directory


In [15]:
import pandas as pd
import numpy as np
import os
import re
from bs4 import BeautifulSoup
from html import unescape
from tqdm import tqdm
from dateutil.parser import parse
import xml.etree.ElementTree as ET

def clean(s):
    regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    s = re.sub(regex, "[REF]", s)
    s = unescape(BeautifulSoup(s,'lxml').get_text())
    if "<br />" in s:
        s=re.sub(r'(\n|<br />)', " ", s)
    else:
        s=re.sub(r'\n', "  ", s)
    s = re.sub(r"\'", "'", s, flags=re.MULTILINE)
    return s

In [16]:
df = pd.read_csv('touche22-task2-stance-dataset.tsv', sep='\t')
i = df[df['ds']=='yahoo'].id.to_list()

In [18]:
df

,ds,id,question,answer,answer_stance,answer_stance_object,object_count,object_1,mask_pos_1,object_2,mask_pos_2
0,softwareengineering.stackexchange,10373,What's the better way to charge for a cloud pl...,"Like all good questions, the answer depends. I...",2,simple but more expensive,2,simple but more expensive,"[[231, 252], [389, 393], [605, 619], [753, 757]]",complicated but cheaper,[]
1,softwareengineering.stackexchange,18838,Haskell AND Lisp vs. Haskell OR Lisp,"I suggest learning both, Haskell first, then C...",2,Haskell AND Lisp,2,Haskell AND Lisp,"[[25, 56]]",Haskell OR Lisp,[]
2,softwareengineering.stackexchange,19392,When is it better to offload work to the RDBMS...,You want to do all set-based operations in the...,3,do it in code,2,offload work to the RDBMS,"[[40, 55], [232, 248]]",do it in code,"[[165, 175], [578, 598]]"
3,softwareengineering.stackexchange,20653,Is it better to specialize in a single field I...,"Specialise if you enjoy it As you are aware, ...",3,expand into other fields to broaden my horizons,2,to specialize in a single field I like,"[[53, 63], [404, 410], [512, 519]]",expand into other fields to broaden my horizons,"[[892, 933]]"
4,softwareengineering.stackexchange,21186,Microsoft SDE Interview vs Microsoft SDET Inte...,"Unfortunately, those are both myths. SDEs and ...",1,Neutral,2,Microsoft SDE Interview,"[[37, 41], [283, 295], [950, 962]]",Microsoft SDET Interview,"[[46, 51], [341, 354], [832, 836], [938, 945]]"
...,...,...,...,...,...,...,...,...,...,...,...
951,yahoo,4268101,4268101,4268101,2,free downloadable browsers,2,free downloadable browsers,"[[73, 80], [227, 234], [480, 487], [739, 746]]",Internet Explorer,"[[775, 777]]"
952,yahoo,4369635,4369635,4369635,1,Neutral,2,second hand smoke,"[[149, 166]]",first hand,[]
953,yahoo,4413933,4413933,4413933,2,Rock,2,Rock,"[[10, 14], [472, 476], [500, 504]]",Rap,"[[23, 34], [572, 575]]"
954,yahoo,618287,618287,618287,3,Ang Lee,2,Tsui Hark,[],Ang Lee,"[[9, 16]]"


In [17]:
%%time

idx=""
question=[]
ids=[]
answers = []
ans_niklas=[]
ans_found=[]
uris=[]
found=""
for file in tqdm(os.listdir("full")): #path to the directory with the split dataset (e.g., the folder is called full)
    print(file)
    if '-' in file:
        tree = ET.parse("full/"+file)
        for r in tree.iter(tag='vespaadd'):
            r = r[0]
            uri=r.findtext('uri')
            if int(uri) in i:
                for idx, row in df.iterrows():
                    if int(uri)==row.id:
                        df.at[idx,'question'] = clean(r.findtext('subject'))
                        df.at[idx,'answer'] = clean(r.findtext('bestanswer'))

FileNotFoundError: [Errno 2] No such file or directory: 'full'

In [5]:
df.keys()

Index(['ds', 'id', 'question', 'answer', 'answer_stance',
       'answer_stance_object', 'object_count', 'object_1', 'mask_pos_1',
       'object_2', 'mask_pos_2'],
      dtype='object')

In [6]:
df = df[['question', 'answer', 'answer_stance_object', 'object_count', 'object_1', 'mask_pos_1', 'object_2', 'mask_pos_2']]

In [7]:
df.head() # result df

,question,answer,answer_stance_object,object_count,object_1,mask_pos_1,object_2,mask_pos_2
0,What's the better way to charge for a cloud pl...,"Like all good questions, the answer depends. I...",simple but more expensive,2,simple but more expensive,"[[231, 252], [389, 393], [605, 619], [753, 757]]",complicated but cheaper,[]
1,Haskell AND Lisp vs. Haskell OR Lisp,"I suggest learning both, Haskell first, then C...",Haskell AND Lisp,2,Haskell AND Lisp,"[[25, 56]]",Haskell OR Lisp,[]
2,When is it better to offload work to the RDBMS...,You want to do all set-based operations in the...,do it in code,2,offload work to the RDBMS,"[[40, 55], [232, 248]]",do it in code,"[[165, 175], [578, 598]]"
3,Is it better to specialize in a single field I...,"Specialise if you enjoy it As you are aware, ...",expand into other fields to broaden my horizons,2,to specialize in a single field I like,"[[53, 63], [404, 410], [512, 519]]",expand into other fields to broaden my horizons,"[[892, 933]]"
4,Microsoft SDE Interview vs Microsoft SDET Inte...,"Unfortunately, those are both myths. SDEs and ...",Neutral,2,Microsoft SDE Interview,"[[37, 41], [283, 295], [950, 962]]",Microsoft SDET Interview,"[[46, 51], [341, 354], [832, 836], [938, 945]]"


In [8]:
for elem in df.question[:5]:
    print (elem)

What's the better way to charge for a cloud platform, simple but more expensive, or complicated but cheaper for most?
Haskell AND Lisp vs. Haskell OR Lisp
When is it better to offload work to the RDBMS rather than to do it in code?
Is it better to specialize in a single field I like, or expand into other fields to broaden my horizons?
Microsoft SDE Interview vs Microsoft SDET Interview and Resources to Study


In [9]:
for elem in df.object_1[:5]:
    print (elem)

simple but more expensive
Haskell AND Lisp
offload work to the RDBMS
to specialize in a single field I like
Microsoft SDE Interview


In [10]:
for elem in df.object_2[:5]:
    print (elem)

complicated but cheaper
Haskell OR Lisp
do it in code
expand into other fields to broaden my horizons
Microsoft SDET Interview


In [11]:
for elem in df.answer[:5]:
    print ("\n")
    print (elem)



Like all good questions, the answer depends. It depends on your application: Is it something you are likely going to be using periodically? A lot on some days of every month and very little the rest of the month? Once a year? Then a pay-as-you-go model would be 'nicer'.  It also depends on the sophistication of your users. Will they bother to do the mental arithmetic if you explain the PAYG model to them? Will they care?  How much will your users be spending? $5? $10? They might simple not care for that amount of money to optimize their experience and would probably rather go for something that is simple or easy.   Could the two models make a differences as large as $1000? Then you might find people are more than willing to have a look at the PAYG terms...  


I suggest learning both, Haskell first, then Common Lisp. My experience with Haskell was that the static typing seemed to be a restricting annoyance at first, but once I got used to it, I noticed that most of my type errors had

In [ ]:
mask_pos_1, mask_pos_2: The list of position of the object_1 / object_2 mentions in the answer (Note: the objects were labeled manually, so that the objects in the answer can be syntactically different from the ones in the question, e.g., synonyms, acronyms, abbreviations, etc.)